In [41]:
import api, preprocessing, similar, gear, graph
from flask import Flask, render_template, request, redirect, url_for
import pymysql
from datetime import datetime
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import io 
from io import BytesIO
import base64

app = Flask(__name__)

def get_db_connection():
    return pymysql.connect(
        host='minseok.mysql.database.azure.com',
        user='minseok',
        password='seok9745@@',
        db='project',
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor,
        ssl={'ca': "./DigiCertGlobalRootCA.crt.pem"},
        ssl_disabled=False
    )


#첫 화면, 라인과 유저네임 받기 
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        Line = request.form.get('Line')
        username = request.form.get('username')

        conn = get_db_connection()
        cursor = conn.cursor()

        #user의 플레이 정보 불러오기
        char_user, GPM, VSPM, DPM, KP, XPD, tier, rank = api.getAPI(username,Line)

        #입력 데이터 전처리 
        cursor.execute("SELECT GPM, VSPM, DPM, KP, XPD FROM usermeandata WHERE TIER = %s AND Line = %s", (tier,Line,))
        average_tier = cursor.fetchall()


        GPM_user, VSPM_user, DPM_user, KP_user, XPD_user = preprocessing.normarization(average_tier, GPM, VSPM, DPM, KP, XPD)


        #프로 선수 데이터 DB에서 가져오기 
    
        cursor.execute("SELECT * FROM pro_game_data WHERE LINE = '%s'", (Line,))
        players = cursor.fetchall()

        similar_player, char_pro, GPM_pro, VSPM_pro, DPM_pro, KP_pro, XPD_pro = similar.pro_player(players, GPM, VSPM, DPM, KP, XPD)


        #그래프 그리기 
        data = {
        'Feature': ['GPM', 'VSPM', 'DPM', 'KP', 'XPD'],
        'User': [GPM_user, VSPM_user, DPM_user, KP_user, XPD_user],
        similar_player: [GPM_pro, VSPM_pro, DPM_pro, KP_pro, XPD_pro]
        }
        plot_df = pd.DataFrame(data)
        url = graph.plot(plot_df,similar_player)


        #장비 가져오기 
        cursor.execute("SELECT Mouse, Keyboard FROM gaming_gear WHERE Player = %s", (similar_player,))
        gear_data = cursor.fetchone()

        mouse = gear_data['Mouse']
        keyboard = gear_data['Keyboard']

        #DB 저장 
        query = """INSERT INTO table3 (username, Line, char_user, similar_player, time) VALUES (%s, %s, %s, %s, %s)"""
        cursor.execute(query, (username, Line, char_user, similar_player,datetime.now()))

        conn.commit()
        cursor.close()
        conn.close()

        return redirect(url_for('result',
                        Line=Line,
                        username=username,
                        char_user=char_user,
                        char_pro = char_pro,
                        similar_player= similar_player,
                        url = url, mouse = mouse, keyboard = keyboard, tier = tier, rank = rank))

    return render_template('index.html')




@app.route("/result")
def result():
    Line = request.args.get('Line', default="", type=str)
    username = request.args.get('username', default="", type=str)
    char_user = request.args.get('char_user', default="", type=str)
    char_pro = request.args.get('char_pro', default="", type=str)
    similar_player = request.args.get('similar_player', default="", type=str)
    mouse = request.args.get('mouse', default="", type=str)
    keyboard = request.args.get('keyboard', default="", type=str)
    url = request.args.get('url', default="", type=str)
    tier = request.args.get('tier', default="", type=str)
    rank = request.args.get('rank', default="", type=str)
    
    





    return render_template("result.html", Line=Line,
                        username=username,
                        char_user=char_user,
                        char_pro = char_pro,
                        similar_player= similar_player,
                        url = url, mouse = mouse, keyboard = keyboard,
                        tier = tier, rank = rank)


if __name__ == "__main__":
    app.run(debug=True)


{'id': 'SuizpWp1ccmGtGLQWeVJ7R68fENOrM6AHnqzSPL0xR3V8z8',
 'accountId': 'uJs5s3fmV398HmIvCTh-Mdl3X04CwNpZMdGvJQcTXybpqIo',
 'puuid': 'oTEgUeI9IsDw1c54uEtwpXi5FvXSbKV8ZmNpfc9G4cnOb4h4O-4J7NJHTd9ILMZimWszesEcoRXmPw',
 'name': '제 재',
 'profileIconId': 1109,
 'revisionDate': 1690214341000,
 'summonerLevel': 239}

In [33]:
n = str(10)
rankUrl = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/' + puuid + '/ids?queue=420&type=ranked&start=0&count='+n+'&api_key='+ apiKey
r = requests.get(rankUrl)
r = r.json()

rankId = r
print(rankId)

['KR_6146459661']


In [34]:

for i in rankId:
    url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + i + '?api_key=' + apiKey
    r = requests.get(url)
    r = r.json()
    info = r['info']  # 전체 데이터에서 info를 추출
    part = info['participants'] # info 데이터에서 유저들의 정보 추출

result = info

In [35]:
df = pd.DataFrame(r['info']['participants'])

In [39]:
df

,assists,baronKills,basicPings,bountyLevel,challenges,champExperience,champLevel,championId,championName,championTransform,...,trueDamageTaken,turretKills,turretTakedowns,turretsLost,unrealKills,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,7,0,23,0,"{'12AssistStreakCount': 0, 'abilityUses': 276,...",14375,15,266,Aatrox,0,...,916,0,1,11,0,15,0,2,4,False
1,8,0,18,0,"{'12AssistStreakCount': 0, 'abilityUses': 640,...",19242,18,120,Hecarim,0,...,1670,1,1,11,0,19,5,2,6,False
2,5,0,6,0,"{'12AssistStreakCount': 0, 'abilityUses': 160,...",16743,17,55,Katarina,0,...,566,0,0,11,0,25,4,5,11,False
3,5,0,20,0,"{'12AssistStreakCount': 0, 'abilityUses': 590,...",17041,17,157,Yasuo,0,...,428,1,1,11,0,22,2,6,10,False
4,13,0,34,0,"{'12AssistStreakCount': 1, 'abilityUses': 194,...",13950,15,117,Lulu,0,...,132,0,0,11,0,82,7,14,36,False
5,15,0,28,4,"{'12AssistStreakCount': 0, 'abilityUses': 307,...",19817,18,106,Volibear,0,...,302,0,2,2,0,21,0,2,10,True
6,15,1,49,1,"{'12AssistStreakCount': 0, 'abilityUses': 1335...",19821,18,76,Nidalee,0,...,1610,2,3,2,0,45,6,8,7,True
7,17,0,40,0,"{'12AssistStreakCount': 0, 'abilityUses': 273,...",20853,18,112,Viktor,0,...,598,3,4,2,0,35,3,7,9,True
8,9,0,21,0,"{'12AssistStreakCount': 0, 'abilityUses': 523,...",18181,17,81,Ezreal,0,...,532,3,3,2,0,22,4,3,8,True
9,18,0,11,1,"{'12AssistStreakCount': 0, 'abilityUses': 274,...",14509,15,43,Karma,0,...,1050,1,1,2,0,93,13,7,56,True


In [37]:
df['championName']

0      Aatrox
1     Hecarim
2    Katarina
3       Yasuo
4        Lulu
5    Volibear
6     Nidalee
7      Viktor
8      Ezreal
9       Karma
Name: championName, dtype: object